In [1]:
from PIL import Image, ImageFilter
import numpy as np


In [20]:
def is_pixel_gray(r, g, b):
    return r == g and g == b

def preprocess_image(path):
    image = Image.open(path)
    width = image.width
    height = image.height
    # correct full masks
    for j in range(width):
        for i in range(height):
            r, g, b = image.getpixel((j, i)) 
            bm = (r == g and g != b) # blue mask
            rm = (g == b and g != r) # red mask
            gm = (r == b and r != g) # green mask
            if bm or rm or gm:
                v = r
                if rm: v = g
                image.putpixel((j,i), (v, v, v))
                
    # for j in range(width):
    #     for i in range(height):
    #         r, g, b = image.getpixel((j, i)) 
    #         bm = b >= g and b >= r # blue mask
    #         rm = r >= g and r >= b # red mask
    #         gm = g >= r and g >= b # green mask
    #         if bm or rm or gm:
    #             if rm: v = (g + b) / 2
    #             if gm: v = (r + b) / 2
    #             if bm: v = (r + g) / 2
    #             v = round(v)
    #             image.putpixel((j,i), (v, v, v))
                
    # image.show()
    ni = np.array(image)
    r = ni[:,:,0]
    g = ni[:,:,1]
    b = ni[:,:,2]
    mask = ((r != g) | (g != b)) * 255
    mask = Image.fromarray(mask.astype(np.uint8))
    filter = ImageFilter.GaussianBlur(radius=6)
    gray = image.convert('L')
    gray_blur = gray.filter(filter)
    prevBlock = None
    prevGray = None
    average_gray = []

    for j in range(width):
        for i in range(height):
            block = mask.getpixel((j, i)) == 255
            current_gray = gray_blur.getpixel((j, i))
            if not block and prevBlock is not None: # leave mask
                midGrayAround = (current_gray + prevGray) / 2
                midGrayIn = sum(average_gray) / len(average_gray)
                delta = midGrayAround - midGrayIn
                for sub_i in range(prevBlock, i):
                    c = gray.getpixel((j, i))
                    new_gray = round(c + delta)
                    gray.putpixel((j, sub_i), new_gray)
                prevBlock = None
                prevGray = None
                average_gray = []
            elif block and prevBlock is None: # enter mask
                prevBlock = i
            
            if block:
                average_gray.append(current_gray)
            
            if not block:
                prevGray = current_gray
            
    gray.show()
    image.show()



In [21]:
preprocess_image("A/A/A3/to/c1r1e0n2-0.png")

kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found
kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found
org.kde.kdegraphics.gwenview.lib: Unresolved mime type  "image/x-mng"
org.kde.kdegraphics.gwenview.lib: Unresolved raw mime type  "image/x-nikon-nrw"
org.kde.kdegraphics.gwenview.lib: Unresolved raw mime type  "image/x-samsung-srw"
